<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Полиномиальная интерполяция

### Modules - Curve Fitting
<section class="post-meta">
By Jonas Tjemsland, Andreas Krogen, Håkon Ånes and Jon Andreas Støvneng
</section>
Last edited: January 26th 2018 
___

В численном анализе интерполяция - это метод (повторного) построения функции между заданным набором точек данных. В физике и науке в целом часто можно получить ряд точек данных из выборок или экспериментов. Если мы знаем (или предполагаем), что эти точки данных верны, мы можем использовать интерполяцию для оценки новых данных в диапазоне этих заданных дискретных точек данных. 

Интерполяция также может использоваться для представления сложных в вычислительном отношении функций в виде более простых функций. Затем, с помощью таблицы из нескольких оценок функций, можно легко аппроксимировать истинную функцию с высокой степенью точности.

Если функция проходит через набор точек данных, считается, что эта функция интерполирует эти точки. В этом модуле мы обсудим интерполяцию многочленами.

Сначала мы импортируем необходимые библиотеки и задаем некоторые общие параметры рисунка.

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import random as rnd
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
# Set common figure parameters
newparams = {
    # Figure
    'figure.figsize': (16, 6), 'axes.grid': True,
    # Lines and points
    'lines.linewidth': 1.5, 'lines.markersize': 10,
    # Fonts
    'font.size': 20, 'mathtext.fontset': 'stix', 'font.family': 'STIXGeneral'
}
plt.rcParams.update(newparams)

## Интерполяция Лагранжа
Для заданного набора точек данных $(x_1,y_1), (x_2,y_2),\cdots, (x_n,y_n)$ легко проверить, что полином степени $n-1$
$$P_{n-1}(x) = \sum_{j=1}^ny_jL_j(x),$$
где
$$L_j(x)=\prod_{i = 0, i\neq j}^n\frac{x-x_i}{x_j-x_i},$$
проходит через все заданные точки данных. $P_{n-1}$ называется интерполирующим полиномом Лагранжа, а $L_j$ - базисными полиномами Лагранжа.

Например, полином Лагранжа второго порядка равен
$$P_2(x)=y_1\frac{(x-x_2)(x-x_3)}{(x_1-x_2)(x_1-x_3)}+y_2\frac{(x-x_1)(x-x_3)}{(x_2-x_1)(x_2-x_3)}+y_3\frac{(x-x_1)(x-x_2)}{(x_3-x_1)(x_3-x_2)}.$$

Можно показать, что если точки данных имеют различные значения $x$, то существует один и только один полином $P$ степени $n-1$ или меньше, такой, что $P(x_i)=y_i$ для $i=1,\cdots,n$. Другими словами, $\textbf{интерполирующий полином уникален}$.

Это можно показать, например, предположив, что $P(x)$ и $Q(x)$ имеют степень $n-1$ и оба интерполируют заданный набор $n$ различных точек данных. Пусть $r(x)=P(x)-Q(x)$. Тогда $r(x)$ является многочленом степени не более $n-1$, и поскольку $P(x_i)-Q(x_i)=0$, $r(x)$ имеет корни $n$. Но, как мы все знаем, многочлен степени $n-1$ имеет не более $n-1$ корней. Таким образом, единственный вывод состоит в том, что $P(x)=Q(x)$.

После того, как все улажено, давайте определим некоторые функции, которые вычисляют интерполяционный полином Лагранжа из заданного набора точек данных.

In [ ]:
def lagrangeBasisPolynomials(x_data,x):
    """Вычисляет базисные полиномы Лагранжа.
    Параметры:
        x_data: float arr. Все x-значения набора данных.
        x: float arr. значения по оси x.
    Возвращается:
        L: float arr. Базисные многочлены Лагранжа. 
    """
    n = len(x_data)
    L = [1]*n
    for i in range(0,n):
        for j in range(0,n):
            if (j != i):
                L[j] = L[j]*(x - x_data[i])/(x_data[j] - x_data[i])
    return L

def lagrangeInterpolation(x_data,y_data,x):
    """Вычисляет интерполяционные полиномы Лагранжа из n точек данных.
    Параметры:
        x_data: float arr. Все x-значения набора данных.
        y_data: float arr. Все y-значения набора данных.
                x_data и y_data должны быть одинакового размера.
        x: float arr. значения по оси x.
    Возвращается:
        P: float arr. Базисные многочлены Лагранжа. 
    """
    L = lagrangeBasisPolynomials(x_data, x)
    P = x*0;
    for i in range(0, len(x_data)):
        P = P + y_data[i]*L[i]
    return P

Давайте попробуем на примере.

In [ ]:
# Определим некоторые точки данных.
x_data = [1.0, 4.0, 7.0, 10.0, 11.0, 2.0]
y_data = [1.2, 1.1,-9.1,  1.0,  1.0, 0.0]

# Определим ось x.
x = np.linspace(min(x_data), max(x_data),100)

# Выполним интерполяцию Лагранжа.
P = lagrangeInterpolation(x_data, y_data, x)

# Построим график результатов.
plt.plot(x, P, label='Interpolation polynomial')
plt.plot(x_data, y_data, '*', label='Data points')
plt.legend(), plt.xlabel('$x$'), plt.ylabel('$y$');

Если бы мы добавили еще одну точку к нашим данным, все вычисления нужно было бы выполнить снова. Это одна из причин того, что интерполяция Лагранжа часто не используется в вычислениях, но она очень интуитивно понятна и проста в реализации.

## Интерполяционные формулы Ньютона
**Newton's divided differences**  
Обозначим уникальный интерполяционный полином для набора точек данных $(x_1,y_1), (x_2,y_2),\cdots, (x_n,y_n)$ как
$$P_{n-1}=f[x_1] + f[x_1\ x_2](x-x_1)+f[x_1\ x_2\ x_3](x-x_1)(x-x_2)+\cdots+f[x_1\ \cdots\ x_n](x-1)\cdots(x-x_{n-1}),$$
где $f[\cdots]$ являются различными коэффициентами многочлена. Эти коэффициенты находятся по формуле [1]
$$f[x_k]=y_k,$$
$$f[x_k \ x_{k+1}]=\frac{f[x_{k+1}]-f[x_k]}{x_{k+1}-x_k},$$
$$f[x_k \ x_{k+1} \ x_{k+2}]=\frac{f[x_{k+1} \ x_{k+2}]-f[x_k \ x_{k+1}]}{x_{k+2}-x_k},$$
$$\vdots$$
Этот алгоритм можно отобразить в таблице, с которой легко обращаться. Для трех точек данных эта таблица

$\qquad\qquad f[x_1]$

$\qquad\qquad\qquad\quad f[x_1 \ x_2]$

$\qquad\qquad f[x_2]\qquad\qquad\quad f[x_1 \ x_2 \ x_3]$

$\qquad\qquad\qquad\quad f[x_2 \ x_3]$

$\qquad\qquad f[x_3]$

Теперь мы можем прочитать коэффициент из первой строки. Когда добавляется еще одна точка данных, нам нужно вычислить только одну новую "строку" треугольника. Теперь определим некоторые функции, которые вычисляют интерполяционный полином с использованием формул Ньютона, а затем снова выполним приведенный выше пример, используя этот метод.

In [ ]:
def newtonsDividedDifference(x_data,y_data):
    """Вычисляет коэффициенты интерполяционного полинома
        из n точек данных, используя Интерполяционные формулы Ньютона.
    Parameters:
        x_data: float arr. Все x-значения набора данных.
        y_data: float arr. Все y-значения набора данных.
                x_data и y_data должна быть одинакового размера.
    Возвращается:
        c:      float arr. Коэффициенты интерполяционного полинома.
    """
    n = len(x_data) - 1
    c = y_data.copy()
    for i in range(1, n + 1):
        for j in range(n, i - 1 , -1):
            c[j] = (c[j] - c[j - 1])/(x_data[j] - x_data[j - i])
    return c

def polyCoeff(x,x_data,c):
    """Возвращает многочлен в форме c_0+c_1*(x-x_1)+c_2*(x-x_1)(x-x_2)...,
    где c_i = c[i] и x_i = x_data[i].
    Параметры:
        x_data: float arr. Все x-значения набора данных.
        c:      float arr. Коэффициенты многочлена.
        x:      float arr. ось x.
    Возвращается:
        P:      float arr. Полином, вычисленный в x.
    """
    n = len(c) - 1
    P = c[n]
    for i in range( n - 1, -1, -1 ):
        P = P*(x - x_data[i]) + c[i]
    return P

In [ ]:
# Perform Newton's divided difference.
c = newtonsDividedDifference(x_data, y_data)
# Write out the interpolating polynomial.
P = polyCoeff(x, x_data, c)

# Plot the results.
plt.figure()
plt.plot(x, P, label='Interpolation polynomial')
plt.plot(x_data, y_data, '*', label='Data points')
plt.legend(), plt.xlabel('$x$'), plt.ylabel('$y$');

## Аппроксимация сложной функции. Оценка ошибок
Мы можем легко применить предыдущие обсуждения для аппроксимации (сложной) функции. Поскольку эта функция известна, мы можем вычислить ошибку.

Попробуем создать приближение к тригонометрической функции
$$f(x)=\cos(x),$$
с использованием набора равномерно распределенных оценок функций в качестве набора данных.

In [ ]:
n = 6 # Количество точек.

# Определит аппроксимируемую функцию.
def f(x): return np.cos(x)
# Определит значения по оси x.
x = np.linspace(-np.pi,np.pi,500)
# Постройт график функции.
plt.plot(x,f(x),'--',label=r'$f(x)=\cos(x)$.')

# Оценки функций.
x_data = np.linspace(-np.pi,np.pi,n)
y_data = f(x_data)

# Выполняет интерполяцию. Можно выбрать любой из предшествующих методов.
# --Интерполяция Лагранжа.
#P = lagrangeInterpolation(x_data,y_data,x)
# --Интерполяционные формулы Ньютона
c = newtonsDividedDifference(x_data,y_data)
P = polyCoeff(x,x_data,c)

# Построение графиков результатов и запись максимальной абсолютной ошибки.
plt.plot(x,P,label='Interpolation polynomial.')
plt.plot(x_data,y_data,'*',label='Data points.')
plt.legend(), plt.xlabel('$x$'), plt.ylabel('$y$');
E = max(abs(f(x)-P)), 
print("Максимальная абсолютная погрешность: E = %.3e." % (E))

## Феномен Рунге.
Попробуем аппроксимировать функцию
$$f(x) = 20\exp\left(-20 x^2\right)$$
в интервале $x\in [-1,1]$ полиномиальной интерполяцией. Начнем с интерполяции из равномерно распределенных оценок функций.

In [ ]:
n = 13 # Количество оценочных точек.

# Определит аппроксимируемую функцию.
def f(x): return 20*np.exp(-20*x**2)

# Определит значения по оси x.
x = np.linspace(-1,1,500)
# Постройт график функции.
plt.plot(x, f(x), '--', label=r'$f(x) = 20\exp\left(-20 x^2\right)$')

# Оценки функций.
x_data = np.linspace(-1, 1, n)
y_data = f(x_data)

# Выполняет интерполяцию. Можно выбрать любой из предшествующих методов.
P = lagrangeInterpolation(x_data, y_data, x)
#c = newtonsDividedDifference(x_data,y_data)
#P = polyCoeff(x,x_data,c)

# Построение графиков результатов и запись максимальной абсолютной ошибки.
plt.plot(x, P, label='Interpolation polynomial')
plt.plot(x_data, y_data, '*', label='Data points')
plt.legend(), plt.xlabel('$x$'), plt.ylabel('$y$');
E = max(abs(f(x)-P)); print("Максимальная абсолютная погрешность: E = %.3e." % (E))

Из приведенного выше рисунка видно, что интерполяционный полином является хорошим приближением к функции, близкой к началу координат. Однако ближе к концу интервала интерполяции мы наблюдаем так называемый феномен Рунге. Чем больше используется оценка функции, тем выше становится ошибка. Попробуйте сами!

Феномена Рунге можно избежать, более разумно выбрав оценку функции (интерполяция Чебышева) или используя так называемую сплайн-интерполяцию (см. Наш блокнот на [Cubic Splines](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/cubic_splines.ipynb)).

## Многочлены Чебышева. Интерполяция Чебышева
Для более полного обсуждения и выводов мы отсылаем вас к учебнику [1], который был использован в качестве основного источника в этом подразделе.

Многочлен Чебышева $n$-го порядка на интервале $x\in [-1,1]$ определяется как
$$T_n(x)=\cos(n \arccos x).$$
Используя некоторые тригонометрические соотношения, можно показать, что
$$T_0 = 1$$
$$T_1 = x$$
$$T_2 = 2x^2-1$$
$$T_3 = 4x^3-3x$$
$$\vdots$$
На самом деле, можно вывести рекурсивную формулу
$$T_{n+1}(x)=2xT_n(x)-T_{n-1}(x).$$
Заметим, что корни многочленов Чебышева
$$x_k = \cos\frac{\pi \left(2k-1\right)}{2n},\quad k=1,2,...,n.$$
и что ведущий коэффициент равен $2^{n-1}$.
Давайте визуализируем эти многочлены.

In [ ]:
n = 5 # Число многочленов.
x = np.linspace(-1,1,100) # ось x.

# Построение многочленов Чебышева.
# --- Используя рекурсивную формулу.
T0 = [1]*len(x)
T1 = x
plt.plot(x, T0, label=r'$n = 0$')
plt.plot(x, T1, label=r'$n = 1$')
for i in range(2, n):
    T2 = 2*x*T1 - T0
    plt.plot(x, T2, label=r"$n = %d$"%(i))
    T0 = T1; T1 = T2
# --- Используя определение.
#for i in range(0,n):
#    plt.plot(x,np.cos(i*np.arccos(x)),label = r"$n = %d$" % i)

# Установит текст рисунка.
plt.legend(), plt.title('Chebyshev polynomials.'), plt.ylim([-1.1, 1.1]);

Для простоты предположим, что мы имеем дело с интерполирующим полиномом на фиксированном интервале $x\in[-1,1]$ (который легко обобщается). 

Можно показать, что если $f(x)$ является $n$ раз непрерывно дифференцируемой, то ошибка интерполяции задается
$$f(x)-P(x)=\frac{(x-x_1)(x-x_2)\cdots (x-x_n)}{n!}f^{(n)}(c),$$
где $c\in [\min(x,x_1,\cdots,x_n),\max(x,x_1,\cdots,x_n)]$ [1].

Ошибка интерполяции сама по себе является полиномом степени $n$ с некоторым максимальным значением на заданном интервале. Наша цель состоит в том, чтобы минимизировать эту ошибку, выбрав точки интерполяции определенным образом. К счастью, **теорема Чебышева** дает ответ.

Точки интерполяции, которые минимизируют $\max_{x\in[-1,1]}|(x-x_1)(x-x_2)\cdots (x-x_n)|$ это  
$$x_i = \cos\frac{(2i-1)\pi}{2n},\quad i = 1,2,\cdots, n,$$
с минимальным значением $1/2^{n-1}$. На самом деле, можно показать, что минимальное значение
$$(x-x_1)(x-x_2)\cdots (x-x_n) = \frac{1}{2^{n-1}}T_n(x).$$

Давайте сделаем набросок доказательства. Пусть $P_n(x)$ - монический многочлен (коэффициент высшей степени равен 1). Предположим, что этот многочлен имеет меньший абсолютный максимум, чем $T_n(x)/2^{n-1}$, который также является моническим многочленом. Другими словами, $|P_n(x)|<|1/2^{n-1}|$, $x\in[-1,1]$. Поскольку $T_n(x)$ чередуется между $-1$ и $1$ $n$ раз, $P_n(x)-T_n(x)/2^{n-1}$ имеет по крайней мере $n$ корней, что означает, что $P_n(x)-T_n(x)/2^{n-1}$ имеет степень не менее $n$. Это противоречие, так как $P_n(x)-T_n(x)/2^{n-1}$ должно иметь степень $n-1$ или меньше, потому что и $P_n(x)$, и $T_n(x)/2^{n-1}$ являются моническими полиномами степени $n$.

Используя узлы Чебышева в качестве баз для интерполяции, ошибка интерполяции будет равномерно распределена по всему интервалу. Следующий код визуализирует обсуждение в этом разделе.

In [ ]:
# Определит аппроксимируемую функцию.
def f(x): return 20*np.exp(-20*x**2)

x = np.linspace(-1, 1, 500) # ось x.
n = 11 # Количество базовых точек.

# Настройка подзаголовка с той же осью x.
# Один для интерполяций и один для соответствующих ошибок.
figs = newparams['figure.figsize']
fig,(interpax,errorax) = plt.subplots(2, 1, sharex=True, figsize=(figs[0], figs[1]*2))
interpax.plot(x, f(x), 'b--',label=r'$f(x) = 20\exp\left(-20 x^2\right)$') # Построение графика функции.

# Интерполяция с использованием равномерно распределенных базовых точек.
x_data = np.linspace(-1,1,n); y_data = f(x_data)
P = lagrangeInterpolation(x_data,y_data,x)
interpax.plot(x, P, 'g', label='Uniformly disturbuted base points')
interpax.plot(x_data, y_data, 'g*', label=' ')
errorax.plot(x, P - f(x), 'g', label='Uniformly disturbuted base points')

# Интерполяция с использованием узлов Чебышева в качестве базовых точек.
x_data = np.cos((2*np.linspace(1, n, n) - 1)*np.pi/(2*n))
y_data = f(x_data)
P = lagrangeInterpolation(x_data, y_data,x)
interpax.plot(x, P, 'm', label='Chebyshev nodes as base points')
interpax.plot(x_data, y_data, 'm*', label=' ')
errorax.plot(x, P - f(x), 'm', label='Chebyshev nodes as base points')
interpax.legend()

# Figure texts.
interpax.set_title('Interpolations.'); errorax.set_title('Absolute interpolation error.')
errorax.set_xlabel('$x$'); interpax.set_ylabel('$y$'); errorax.set_ylabel('Absolute error'); 

Если мы хотим использовать базовые точки Чебышева для общего интервала $x\in[a,b]$, точки интерполяции 
$$x_i = \frac{b+a}{2}+\frac{b-a}{2}\cos\frac{(2i-1)\pi}{2n},\quad i = 1,2,\cdots, n.$$

## Интерполяция параметризованных кривых.
Предыдущие обсуждения можно легко обобщить. Например, если мы хотим интерполировать кривую $\vec C(t),\; 0\leq t \leq 1$ по набору данных $(x_1,y_1,z_2), (x_2,y_2,z_2),..., (x_n,y_n,z_n)$, то можно использовать
$$\vec P_{n-1}(t) = \sum_{j=1}^n\vec x_jL_j(t),$$
где
$$L_j(t)=\prod_{i = 0, i\neq j}^n\frac{t-t_i}{t_j-t_i}.$$
Дискуссии о феномене Рунге и узлах Чебышева справедливы для значения параметра $t$, но тогда мы должны выбрать $a$ и $b$ как
$$a = \frac{\cos\left(\frac{\pi}{2n}\right)-1}{2\cos\left(\frac{\pi}{2n}\right)},\quad b = \frac{\cos\left(\frac{\pi}{2n}\right)+1}{2\cos\left(\frac{\pi}{2n}\right)},$$
(а не 0 и 1, как можно было подумать).
В следующем коде как равномерно распределенные узлы параметров, так и узлы параметров Чебышева ($t_i$) используются для интерполяции двух заданных наборов (произвольных) точек данных.

In [ ]:
# Определит некоторые точки данных.
x_data = [-0.5,-1.0,-0.5, 0.2, 1.5, 2.0, 1.0]
y_data = [ 5.0, 3.7, 1.0, 1.0,-0.5, 1.5, 4.0]
n = len(x_data) # Количество точек данных.

# Значения параметра
t = np.linspace(0, 1, 100)

# Интерполяция кривой с использованием равномерно распределенных узлов параметров.
t_i = np.linspace(0, 1, n) # Равномерно распределенные узлы параметров.
Px = lagrangeInterpolation(t_i, x_data, t)
Py = lagrangeInterpolation(t_i, y_data, t)
plt.plot(Px, Py, 'g', label='Uniformly disturbuted nodes') # Построит график результата.

# Интерполяция кривой с использованием узлов Чебышева в качестве узлов параметров.
a = (np.cos(np.pi/(2*n)) - 1)/(2*np.cos(np.pi/(2*n)))
b = (np.cos(np.pi/(2*n)) + 1)/(2*np.cos(np.pi/(2*n)))
t_i = (b + a)/2 + (b - a)/2*np.cos((2*n - 2*np.linspace(1, n, n) + 1)*np.pi/(2*n)) # Узлы Чебышева.
Px = lagrangeInterpolation(t_i, x_data, t)
Py = lagrangeInterpolation(t_i, y_data, t)
plt.plot(Px, Py, 'm', label='Chebyshev nodes') # Построит график результата.

# Построит точки данных и установит текст рисунка.
plt.plot(x_data, y_data, 'r*', label='Data points')
plt.title('Polynomial curve interpolation of a set data points')
plt.legend(), plt.xlabel('$x$'), plt.ylabel('$y$');

Как и ранее, мы также можем аппроксимировать (сложную) кривую. Попробуем аппроксимировать кривую
$$\vec C(t)=\left(\frac{r^2-3}{r^2+1},\frac{r^3-3r}{r^2+1}\right), \quad r\in[-5/2,5/2],$$
используя как равномерно распределенные базовые узлы, так и базовые узлы Чебышева.

In [ ]:
# Определит аппроксимируемую кривую.
# В качестве небольшого трюка мы рассчитаем с 0<t<1 и в определении функции преобразуем в r.
def C(t):
    r = 5*(t - 0.5)
    return (r**2 - 3)/(r**2 + 1), (r**3 - 3*r)/(r**2 + 1)

n = 15 # Количество точек данных.
t = np.linspace(0, 1, 1000) # Значение параметра.

# рисует f(x).
x,y = C(t)
plt.plot(x, y,
 label=r'$\vec C(t)=\left(\frac{r^2-3}{r^2+1},\frac{r^3-3r}{r^2+1}\right), \quad r\in[-5/2,5/2]$')

# Интерполяция кривой с использованием равномерно распределенных узлов параметров.
t_i = np.linspace(0, 1, n) # Равномерно распределенные узлы параметров.
x_data,y_data = C(t_i)
Px = lagrangeInterpolation(t_i, x_data, t)
Py = lagrangeInterpolation(t_i, y_data, t)
plt.plot(Px, Py, 'g', label='Uniformly disturbuted nodes')
plt.plot(x_data, y_data, 'g*', label=' ')
# Вычисление максимальной абсолютной ошибки.
E = max(abs(((Px - x)**2 + (Py - y)**2)**0.5))
print("Равномерно распределенные узлы: E = %.3e." % (E))

# Интерполяция кривой с использованием узлов Чебышева в качестве узлов параметров.
a = (np.cos(np.pi/(2*n))-1)/(2*np.cos(np.pi/(2*n)))
b = (np.cos(np.pi/(2*n))+1)/(2*np.cos(np.pi/(2*n)))
t_i = (b + a)/2 + (b - a)/2*np.cos((2*n-2*np.linspace(1, n, n) + 1)*np.pi/(2*n))
x_data,y_data = C(t_i)
Px = lagrangeInterpolation(t_i, x_data, t)
Py = lagrangeInterpolation(t_i, y_data, t)
plt.plot(Px, Py, 'm', label='Chebyshev nodes')
plt.plot(x_data, y_data, 'm*', label=' ')
# Вычисление максимальной абсолютной ошибки.
E = max(abs(((Px-x)**2+(Py-y)**2)**0.5))
print("Chebyshev nodes: E = %.3e."%(E))

# Установит текст рисунка.
plt.title('Approximation of a curve using polynomial interpolation')
plt.legend(), plt.xlabel('$x$'), plt.ylabel('$y$'), plt.axis('equal');

### Источники для дальнейшего чтения
[1] Sauer, T.: Numerical Analysis international edition, second edition, Pearson 2014

[2] Weisstein, Eric W. "Newton's Divided Difference Interpolation Formula." From MathWorld--A Wolfram Web Resource. http://mathworld.wolfram.com/NewtonsDividedDifferenceInterpolationFormula.html (acquired March 28. 2016)

[3] Archer, Branden and Weisstein, Eric W. "Lagrange Interpolating Polynomial." From MathWorld--A Wolfram Web Resource. http://mathworld.wolfram.com/LagrangeInterpolatingPolynomial.html (acquired March 28. 2016)

Scipy has a number of different interpolation functions. Check it out:
http://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html (acquired March 18. 2016)